# sub-band CSTP with data_set data

In [1]:
import os, glob, sys, platform
import pylab
import scipy.io
from scipy.signal import butter, lfilter, freqs
from sklearn.lda import LDA
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics 
import pdb
import mne
from mne.decoding import CSP
from sklearn import svm
from sklearn.svm import SVC
from sklearn import cross_validation
from sklearn.linear_model import LogisticRegression
import sklearn.preprocessing
from sklearn.decomposition import PCA
%pylab qt

/Users/Villa/anaconda/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/Users/Villa/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""


Populating the interactive namespace from numpy and matplotlib


/Users/Villa/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/Users/Villa/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/Users/Villa/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  """
/Users/Villa/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:114: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata 

In [2]:
# ----------------------------------------------import data---------------------------------------------
def load_epoch(path_data,file_name):
    os.chdir(path_data)
    data=scipy.io.loadmat(file_name)
    keys=data.keys()
    for i in range(0,len(keys)):
        if keys[i] != '__version__' and keys[i] !='__header__' and keys[i] !='__globals__':
            EEG=data[keys[i]]
    return EEG

# ---------------------------------------common spatial temporal pattern------------------------------------
def myCSP(epoch,label):
    data=transpose(epoch,(0,2,1))  # obs,chan,time
    csp=CSP(n_components=6,reg='oas')
    csp.fit(data,label)
    cs_pattern=csp.patterns_.T  # columns are common spatial patterns (chan * patterns)
    cs_filter=csp.filters_  # patterns * chan
    return cs_pattern, cs_filter

def myCTP(epoch,label,downsampling):
    # resampling
    data=epoch[:,0::downsampling,:]
    low=float(300)
    high=float(500)
    ctp=CSP(n_components=6,reg='oas')
    ctp.fit(data,label)
    ct_pattern=ctp.patterns_.T  # columns are common spatial patterns (time * patterns)
    ct_filter=ctp.filters_  # patterns * time
    return ct_pattern, ct_filter

def signal_projection(filter_para, epoch):
    data=transpose(epoch,(0,2,1))   # data: obs,chan,var
    signal=reshape(filter_para.dot(data),(shape(epoch)[0],shape(epoch)[1]))
    return signal

# --------------------------------------------------CSP and CTP comparision--------------------------------
def chan_corre(epoch):
    obs,vari,chan=shape(epoch)[0], shape(epoch)[1], shape(epoch)[2]
    covariance=zeros([chan,chan])
    for i in range(0,obs):
        covariance=covariance+cov(epoch[i,:,:].T)/trace(cov(epoch[i,:,:].T))
    return covariance/obs

def max_target_lamba_ind(epoch_target,epoch_standard,cs_pattern,cs_filter):
    sig_target=chan_corre(epoch_target)
    sig_standard=chan_corre(epoch_standard)
    eig_target=diag(cs_filter.dot(sig_target).dot(cs_filter.T))
    eig_standard=diag(cs_filter.dot(sig_standard).dot(cs_filter.T))
    ind=where(eig_target/eig_standard==max(eig_target/eig_standard))[0]
    return ind[0]

def CSP_filter_sign(cs_filter_para,target):
    cs_target=zeros([1,shape(target)[1]])
    for i in range(0,shape(target)[0]):
        cs_target=cs_target+cs_filter_para.dot(target[i,:,:].T)
    cs_target_mean=cs_target/(i+1)
    if sum(cs_target_mean[:,325:450],axis=1)<0:
        cs_filter_para=-cs_filter_para
    return cs_filter_para

def CTP_pattern_sign(ct_pattern_para):
    if amax(ct_pattern_para)<0:
        ct_pattern_para=-ct_pattern_para
    return ct_pattern_para

# --------------------------------------------------------filter--------------------------------------------
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandpass')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_bandstop(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='bandstop')
    return b, a

def butter_bandstop_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandstop(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y




# --------------------------------------------------classification----------------------------------------------
def covariance(X):
    mu=mean(X,axis=0)
    N=shape(X)[0]
    S=(X-mu).T.dot(X-mu)/(N-1)
    return S

# new lda
def my_new_lda(X_train,label_train):
    x1,x2,label1,label2=sample_arrange(X_train,label_train)
    x1_mean=mean(x1,axis=0,keepdims=True)
    x2_mean=mean(x2,axis=0,keepdims=True)
    s1=zeros([shape(x1)[1],shape(x1)[1]])
    s2=zeros([shape(x2)[1],shape(x2)[1]])
    
    for i in range(0,shape(x1)[0]):
        s1=s1+(x1[i:i+1,:]-x1_mean).T.dot(x1[i:i+1,:]-x1_mean)
        
    for i in range(0,shape(x2)[0]):
        s2=s2+(x2[i:i+1,:]-x2_mean).T.dot(x2[i:i+1,:]-x2_mean)
    sw=(s1+s2)/(shape(X_train)[0]-2)

    eigvalue_sw,eigvector_sw=eig(sw)
    eigmean=mean(eigvalue_sw)
    
    for i in range(0,len(eigvalue_sw)):
        if eigvalue_sw[i]<eigmean:
            eigvalue_sw[i]=eigmean
      
    sw_new=eigvector_sw.dot(diag(eigvalue_sw)).dot(eigvector_sw.T)
    
    w=inv(sw_new).dot((x1_mean-x2_mean).T)
    w0=(x1_mean+x2_mean).dot(w)/2.
    return w, w0

def lda_predict(X_test,label_test,w_lda,w_lda0):
    X_test_lda=lda_out(w_lda,X_test)
    y=X_test_lda-w_lda0 
    out=zeros([shape(label_test)[0],1])
    ind1=where(y>=0)
    ind2=where(y<0)
    out[ind1],out[ind2]=1,0
    return out

def lda_out(w,X_train):
    return X_train.dot(w)

def sample_arrange(X,label):
    x1=X[where(label==1)[0],:]
    x2=X[where(label==0)[0],:]
    label1=label[where(label==1)[0],:]
    label2=label[where(label==0)[0],:]
    return x1,x2,label1,label2


def confusion_matrix(out,target_num):
    ind_TP=where(out[0:target_num]==1)[0]
    ind_FP=where(out[target_num:]==1)[0]
    ind_TN=where(out[target_num:]==0)[0]
    ind_FN=where(out[0:target_num]==0)[0]
    TP=len(ind_TP)
    FP=len(ind_FP)
    TN=len(ind_TN)
    FN=len(ind_FN)
    P=len(out[0:target_num])
    N=len(out[target_num:])
    return float(TP), float(FP), float(TN), float(FN), float(P), float(N)

def balanced_accuracy(TP,FP,TN,FN):
    BA=.5*(TP/(TP+FN)+TN/(TN+FP))
    return BA

def linear_out(X_test,label_test,y):
    out=zeros([shape(label_test)[0],1])
    ind1=where(y>=0)
    ind2=where(y<0)
    out[ind1],out[ind2]=1,0
    return out


def my_bayes(X,label):
    x1,x2,label1,label2=sample_arrange(X,label)
    mv1,sig1=mean(x1,axis=0),covariance(x1)
    mv2,sig2=mean(x2,axis=0),covariance(x2)
    
#     eigvalue_sig1,eigvector_sig1=eig(sig1)
#     eigmean_sig1=mean(eigvalue_sig1)
#     for i in range(0,len(eigvalue_sig1)):
#         if eigvalue_sig1[i]<eigmean_sig1:
#             eigvalue_sig1[i]=eigmean_sig1
#     sig1=eigvector_sig1.dot(diag(eigvalue_sig1)).dot(eigvector_sig1.T)
    
#     eigvalue_sig2,eigvector_sig2=eig(sig2)
#     eigmean_sig2=mean(eigvalue_sig2)
#     for i in range(0,len(eigvalue_sig2)):
#         if eigvalue_sig2[i]<eigmean_sig2:
#             eigvalue_sig2[i]=eigmean_sig2
#     sig2=eigvector_sig2.dot(diag(eigvalue_sig2)).dot(eigvector_sig2.T)
    
    
    pro1=float(len(x1))/(len(x1)+len(x2))
    pro2=float(len(x2))/(len(x1)+len(x2))
    y1=M_pdf(x1,mv1,sig1)
    y2=M_pdf(x2,mv2,sig2)
    return mv1,sig1,pro1,mv2,sig2,pro2

def my_bayes_score(X_test,X_train,label_train):  
    mv1,sig1,pro1,mv2,sig2,pro2=my_bayes(X_train,label_train)
    y1=M_pdf(X_test,mv1,sig1)
    y2=M_pdf(X_test,mv2,sig2)
    p1=pro1*y1/EVI(pro1,y1,pro2,y2)
    p2=pro2*y2/EVI(pro1,y1,pro2,y2)
    return p1,p2

def EVI(pro1,y1,pro2,y2):
    return y1*pro1+y2*pro2

def M_pdf(X,mv,sig):
    # X: obs*feature
    m,n=shape(X)[0],shape(X)[1]
    n=float(n)
    exponent=zeros([m,1])
    y=zeros([m,1])
    for i in range(0,m):
        exponent[i,:]=exp( -1./2.*(X[i:i+1,:]-mv).dot(inv(sig)).dot((X[i:i+1,:]-mv).T))
        y[i,:]=1./(pow( (2*pi),n/2)*(pow (det(sig),1./2.) ) )*(exponent[i,:])
    return y

def bayes_predict(X_test,label_test,X_train,label_train,pstar):
    p1,p2=my_bayes_score(X_test,X_train,label_train)
    out=zeros([shape(label_test)[0],1])
    ind1=where(p1>=pstar)
    ind2=where(p1<pstar)
    out[ind1],out[ind2]=1,0
    return out


def log_var(epoch):
    # epoch: obs*vari
    return log10(var(epoch,axis=1,keepdims=True))

def plot_style():
    font = {'family' : 'normal', 'weight' : 'bold', 'size'   : 14}
    rc('font', **font)
    rc('axes', linewidth=4)
    rc('lines', lw=4)
plot_style()

In [3]:
if platform.system()=='Windows':
    # expert user1 10Hz
    expert_user1_10Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 10Hz\\police'
    expert_user1_10Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 10Hz\\nonsmoking'
    expert_user1_10Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 10Hz\\figure'

    # expert user2 10Hz
    expert_user2_10Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user2 10Hz\\police'
    expert_user2_10Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user2 10Hz\\nonsmoking'
    

    # expert user1 5Hz
    expert_user1_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 5Hz\\figure'
    expert_user1_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 5Hz\\nonsmoking'
    expert_user1_5Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 5Hz\\police'
    expert_user1_5Hz_guy='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 5Hz\\guy'
    expert_user1_5Hz_parking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user1 5Hz\\parking'


    # expert user2 5Hz
    expert_user2_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user2 5Hz\\nonsmoking'
    expert_user2_5Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user2 5Hz\\police'
    expert_user2_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user2 5Hz\\figure'
    

    # expert user5 5Hz
    expert_user5_5Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user5 5Hz\\police'
    expert_user5_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user5 5Hz\\nonsmoking'
    expert_user5_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\expert user5 5Hz\\figure'
    

    # nonexpert user3 10 Hz
    nonexpert_user3_10Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user3 10Hz\\figure'
    nonexpert_user3_10Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user3 10Hz\\police'
    nonexpert_user3_10Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user3 10Hz\\nonsmoking'
    

    # nonexpert user4 5Hz
    nonexpert_user4_5Hz_guy='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user4 5Hz\\guy'
    nonexpert_user4_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user4 5Hz\\figure'
    nonexpert_user4_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user4 5Hz\\nonsmoking'
    

    # nonexpert user6 5Hz
    nonexpert_user6_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user6 5Hz\\figure'
    nonexpert_user6_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user6 5Hz\\nonsmoking'
    nonexpert_user6_5Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user6 5Hz\\police'
    

    # nonexpert user7 5Hz
    nonexpert_user7_5Hz_figure='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user7 5Hz\\figure'
    nonexpert_user7_5Hz_nonsmoking='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user7 5Hz\\nonsmoking'
    nonexpert_user7_5Hz_guy='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user7 5Hz\\guy'
    nonexpert_user7_5Hz_police='D:\\application\\google cloud\\workspace\\cost_leanrning_lda\\data_set\\nonexpert user7 5Hz\\police'
    
elif platform.system()=='Darwin':
    expert_user1_10Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 10Hz/police'
    expert_user1_10Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 10Hz/nonsmoking'
    expert_user1_10Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 10Hz/figure'

    expert_user2_10Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user2 10Hz/police'
    expert_user2_10Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user2 10Hz/nonsmoking'

    expert_user1_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 5Hz/figure'
    expert_user1_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 5Hz/nonsmoking'
    expert_user1_5Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 5Hz/police'
    expert_user1_5Hz_guy='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 5Hz/guy'
    expert_user1_5Hz_parking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user1 5Hz/parking'

    expert_user2_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user2 5Hz/nonsmoking'
    expert_user2_5Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user2 5Hz/police'
    expert_user2_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user2 5Hz/figure'

    expert_user5_5Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user5 5Hz/police'
    expert_user5_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user5 5Hz/nonsmoking'
    expert_user5_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/expert user5 5Hz/figure'

    nonexpert_user3_10Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user3 10Hz/figure'
    nonexpert_user3_10Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user3 10Hz/police'
    nonexpert_user3_10Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user3 10Hz/nonsmoking'

    nonexpert_user4_5Hz_guy='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user4 5Hz/guy'
    nonexpert_user4_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user4 5Hz/figure'
    nonexpert_user4_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user4 5Hz/nonsmoking'

    nonexpert_user6_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user6 5Hz/figure'
    nonexpert_user6_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user6 5Hz/nonsmoking'
    nonexpert_user6_5Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user6 5Hz/police'

    nonexpert_user7_5Hz_figure='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user7 5Hz/figure'
    nonexpert_user7_5Hz_nonsmoking='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user7 5Hz/nonsmoking'
    nonexpert_user7_5Hz_guy='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user7 5Hz/guy'
    nonexpert_user7_5Hz_police='/Users/Villa/Google Drive/workspace/cost_leanrning_lda/data_set/nonexpert user7 5Hz/police'

In [5]:
data_path=[expert_user1_10Hz_police,expert_user1_10Hz_nonsmoking,expert_user1_10Hz_figure,\
          expert_user2_10Hz_police,expert_user2_10Hz_nonsmoking,\
          expert_user1_5Hz_figure,expert_user1_5Hz_nonsmoking,expert_user1_5Hz_police,expert_user1_5Hz_guy,expert_user1_5Hz_parking,\
          expert_user2_5Hz_nonsmoking,expert_user2_5Hz_police,expert_user2_5Hz_figure,\
          expert_user5_5Hz_police,expert_user5_5Hz_nonsmoking,expert_user5_5Hz_figure,\
          nonexpert_user3_10Hz_figure,nonexpert_user3_10Hz_nonsmoking,nonexpert_user3_10Hz_police,\
          nonexpert_user4_5Hz_guy,nonexpert_user4_5Hz_figure,nonexpert_user4_5Hz_nonsmoking,\
          nonexpert_user6_5Hz_figure,nonexpert_user6_5Hz_nonsmoking,nonexpert_user6_5Hz_police,\
          nonexpert_user7_5Hz_figure,nonexpert_user7_5Hz_nonsmoking,nonexpert_user7_5Hz_guy,nonexpert_user7_5Hz_police]
 
data_sub1=[expert_user1_10Hz_police,expert_user1_10Hz_nonsmoking,expert_user1_10Hz_figure] 
data_sub2=[expert_user2_10Hz_police,expert_user2_10Hz_nonsmoking]
data_sub3=[expert_user1_5Hz_figure,expert_user1_5Hz_nonsmoking,expert_user1_5Hz_police] #,expert_user1_5Hz_guy,expert_user1_5Hz_parking
data_sub4=[expert_user2_5Hz_nonsmoking,expert_user2_5Hz_police,expert_user2_5Hz_figure]
data_sub5=[expert_user5_5Hz_police,expert_user5_5Hz_nonsmoking,expert_user5_5Hz_figure]
data_sub6=[nonexpert_user3_10Hz_figure,nonexpert_user3_10Hz_nonsmoking,nonexpert_user3_10Hz_police]
data_sub7=[nonexpert_user4_5Hz_guy,nonexpert_user4_5Hz_figure,nonexpert_user4_5Hz_nonsmoking]
data_sub8=[nonexpert_user6_5Hz_figure,nonexpert_user6_5Hz_nonsmoking,nonexpert_user6_5Hz_police]
data_sub9=[nonexpert_user7_5Hz_figure,nonexpert_user7_5Hz_nonsmoking,nonexpert_user7_5Hz_police] #,nonexpert_user7_5Hz_guy
data_sub=[data_sub1,data_sub2,data_sub3,data_sub4,data_sub5,data_sub6,data_sub7,data_sub8,data_sub9]

# sub-band CSTP cost learning training and testing 

In [7]:
# 3000 tirals in each training and testing, separate train and testing data set 
target_AC_linear_train=[]
standard_AC_linear_train=[]
BA_linear_train=[]

target_AC_lda_train=[]
standard_AC_lda_train=[]
BA_lda_train=[]
lda_project=[]

target_AC_svm_train=[]
standard_AC_svm_train=[]
BA_svm_train=[]

target_AC_bayes_train=[]
standard_AC_bayes_train=[]
BA_bayes_train=[]

target_AC_linear_test=[]
standard_AC_linear_test=[]
BA_linear_test=[]

target_AC_lda_test=[]
standard_AC_lda_test=[]
BA_lda_test=[]

target_AC_svm_test=[]
standard_AC_svm_test=[]
BA_svm_test=[]

target_AC_bayes_test=[]
standard_AC_bayes_test=[]
BA_bayes_test=[]
# len(data_sub)
for i in range(0,9):
    variable,chan=shape(load_epoch(data_path[0],'epoch_target'))[1], 32
    dataset=data_sub[i]
    percent=.9  # training rate
    test_percent=1-percent # testing rate
    portion=.99 # base rate in training session
    
    epoch_target_dataset=[]
    epoch_standard_dataset=[]
    epoch_target=zeros([1,variable,chan])
    epoch_standard=zeros([1,variable,chan])
    for j in range(0,len(dataset)):
        epoch_target_dataset.append(load_epoch(dataset[j],'epoch_target'))
        epoch_standard_dataset.append(load_epoch(dataset[j],'epoch_standard')) # chan variable obs
        epoch_target_32,epoch_standard_32=epoch_target_dataset[j], epoch_standard_dataset[j]
        epoch_target_6,epoch_standard_6=epoch_target_32[:,:,:],\
                                        epoch_standard_32[:,:,:]
#         epoch_target_6,epoch_standard_6=epoch_target_32[array([2,24,13,14,19,17,16,18])-1,:,:],\
#                                         epoch_standard_32[array([2,24,13,14,19,17,16,18])-1,:,:]
        epoch_target_6,epoch_standard_6=epoch_target_6.T, epoch_standard_6.T    # obs variable chan
        epoch_target=r_[epoch_target, epoch_target_6]
        epoch_standard=r_[epoch_standard, epoch_standard_6]
    epoch_target=epoch_target[1:,:,:]    
    epoch_standard=epoch_standard[1:,:,:]

    flag=shape(epoch_target)[0]+shape(epoch_standard)[0] # total number of epochs
    num_t=shape(epoch_target)[0] # number of target
    num_s=shape(epoch_standard)[0] # number of standard
    epoch_all= r_[epoch_target,epoch_standard]
     
#   ----------------------------------------------filter-------------------------------------
#   0-5Hz  
    epoch_all_0_5=zeros_like(epoch_all)
    for j in range(0,chan):
        epoch_all_0_5[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], .1, 5, 250, order=3)

    
#   5-10Hz  
    epoch_all_5_10=zeros_like(epoch_all)
    for j in range(0,chan):
        epoch_all_5_10[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], 5, 10, 250, order=3)   
    
#   10-15Hz  
    epoch_all_10_15=zeros_like(epoch_all)
    for j in range(0,chan):
        epoch_all_10_15[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], 10, 15, 250, order=3)    


#   --------------------------------------------construct sub band training and testing epochs for CSTP--------------------------------
    # 0-5Hz
    epoch_target_train_0_5, epoch_target_test_0_5=cross_validation.train_test_split(epoch_all_0_5[0:num_t,:,:], train_size=percent,random_state=42)
    epoch_standard_train_0_5, epoch_standard_test_0_5=cross_validation.train_test_split(epoch_all_0_5[num_t:,:,:], train_size=percent,random_state=42)
    epoch_train_0_5=r_[epoch_target_train_0_5,epoch_standard_train_0_5]
    label_train=r_[ones(shape(epoch_target_train_0_5)[0]),zeros(shape(epoch_standard_train_0_5)[0])]
    epoch_test_0_5=r_[epoch_target_test_0_5,epoch_standard_test_0_5]
    label_test=r_[ones(shape(epoch_target_test_0_5)[0]),zeros(shape(epoch_standard_test_0_5)[0])]
    
    # 5-10Hz
    epoch_target_train_5_10, epoch_target_test_5_10=cross_validation.train_test_split(epoch_all_5_10[0:num_t,:,:], train_size=percent,random_state=42)
    epoch_standard_train_5_10, epoch_standard_test_5_10=cross_validation.train_test_split(epoch_all_5_10[num_t:,:,:], train_size=percent,random_state=42)
    epoch_train_5_10=r_[epoch_target_train_5_10,epoch_standard_train_5_10]
    epoch_test_5_10=r_[epoch_target_test_5_10,epoch_standard_test_5_10]
    
    # 10-15Hz
    epoch_target_train_10_15, epoch_target_test_10_15=cross_validation.train_test_split(epoch_all_10_15[0:num_t,:,:], train_size=percent,random_state=42)
    epoch_standard_train_10_15, epoch_standard_test_10_15=cross_validation.train_test_split(epoch_all_10_15[num_t:,:,:], train_size=percent,random_state=42)
    epoch_train_10_15=r_[epoch_target_train_10_15,epoch_standard_train_10_15]
    epoch_test_10_15=r_[epoch_target_test_10_15,epoch_standard_test_10_15]
    

    
    
    # ---------------------------------------------CSTP calculation----------------------------------------
    # train 300-500
    downsampling=5
    # 0-5Hz
    cs_pattern_0_5,cs_filter_0_5=myCSP(epoch_train_0_5[:,300:500,:],label_train)
    ct_pattern_0_5, ct_filter_0_5=myCTP(epoch_train_0_5[:,300:500,:],label_train,downsampling)

    ind_eig_cs1_0_5=max_target_lamba_ind(epoch_train_0_5[0:num_t*percent,300:500,:],epoch_train_0_5[num_t*percent:,300:500,:],cs_pattern_0_5,cs_filter_0_5)
    ind_eig_ct1_0_5=max_target_lamba_ind(transpose(epoch_train_0_5[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_0_5[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_0_5,ct_filter_0_5)

    ind_eig_cs2_0_5=max_target_lamba_ind(epoch_train_0_5[num_t*percent:,300:500,:],epoch_train_0_5[0:num_t*percent,300:500,:],cs_pattern_0_5,cs_filter_0_5)
    ind_eig_ct2_0_5=max_target_lamba_ind(transpose(epoch_train_0_5[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_0_5[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_0_5,ct_filter_0_5)
    

    # 5-10Hz
    cs_pattern_5_10,cs_filter_5_10=myCSP(epoch_train_5_10[:,300:500,:],label_train)
    ct_pattern_5_10, ct_filter_5_10=myCTP(epoch_train_5_10[:,300:500,:],label_train,downsampling)

    ind_eig_cs1_5_10=max_target_lamba_ind\
                    (epoch_train_5_10[0:num_t*percent,300:500,:],epoch_train_5_10[num_t*percent:,300:500,:],cs_pattern_5_10,cs_filter_5_10)
    ind_eig_ct1_5_10=max_target_lamba_ind(transpose(epoch_train_5_10[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_5_10[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_5_10,ct_filter_5_10)

    ind_eig_cs2_5_10=max_target_lamba_ind\
                    (epoch_train_5_10[num_t*percent:,300:500,:],epoch_train_5_10[0:num_t*percent,300:500,:],cs_pattern_5_10,cs_filter_5_10)
    ind_eig_ct2_5_10=max_target_lamba_ind(transpose(epoch_train_5_10[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_5_10[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_5_10,ct_filter_5_10)
    
    
    # 10-15Hz
    cs_pattern_10_15,cs_filter_10_15=myCSP(epoch_train_10_15[:,300:500,:],label_train)
    ct_pattern_10_15, ct_filter_10_15=myCTP(epoch_train_10_15[:,300:500,:],label_train,downsampling)

    ind_eig_cs1_10_15=max_target_lamba_ind\
                      (epoch_train_10_15[0:num_t*percent,300:500,:],epoch_train_10_15[num_t*percent:,300:500,:],cs_pattern_10_15,cs_filter_10_15)
    ind_eig_ct1_10_15=max_target_lamba_ind(transpose(epoch_train_10_15[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_10_15[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_10_15,ct_filter_10_15)

    ind_eig_cs2_10_15=max_target_lamba_ind\
                     (epoch_train_10_15[num_t*percent:,300:500,:],epoch_train_10_15[0:num_t*percent,300:500,:],cs_pattern_10_15,cs_filter_10_15)
    ind_eig_ct2_10_15=max_target_lamba_ind(transpose(epoch_train_10_15[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                                  transpose(epoch_train_10_15[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_10_15,ct_filter_10_15)
    
    

    
    
    # -------------------------------------------CSTP training feature selection-----------------------------------
    # CSP feature
    # 0-5Hz
    cs_filter_para_eig1_0_5=cs_filter_0_5[ind_eig_cs1_0_5,:]
    cs_filter_para_eig2_0_5=cs_filter_0_5[ind_eig_cs2_0_5,:]
    epoch_cs_filter_eig1_0_5=signal_projection(cs_filter_para_eig1_0_5, epoch_train_0_5[:,300:500,:])
    epoch_cs_filter_eig2_0_5=signal_projection(cs_filter_para_eig2_0_5, epoch_train_0_5[:,300:500,:])
    target_cs_filter_eig1_0_5,target_cs_filter_eig2_0_5=epoch_cs_filter_eig1_0_5[0:num_t*percent,:], epoch_cs_filter_eig2_0_5[0:num_t*percent,:]
    standard_cs_filter_eig1_0_5,standard_cs_filter_eig2_0_5=epoch_cs_filter_eig1_0_5[num_t*percent:,:], epoch_cs_filter_eig2_0_5[num_t*percent:,:]
    
    # 5-10Hz
    cs_filter_para_eig1_5_10=cs_filter_5_10[ind_eig_cs1_5_10,:]
    cs_filter_para_eig2_5_10=cs_filter_5_10[ind_eig_cs2_5_10,:]
    epoch_cs_filter_eig1_5_10=signal_projection(cs_filter_para_eig1_5_10, epoch_train_5_10[:,300:500,:])
    epoch_cs_filter_eig2_5_10=signal_projection(cs_filter_para_eig2_5_10, epoch_train_5_10[:,300:500,:])
    target_cs_filter_eig1_5_10,target_cs_filter_eig2_5_10=epoch_cs_filter_eig1_5_10[0:num_t*percent,:], epoch_cs_filter_eig2_5_10[0:num_t*percent,:]
    standard_cs_filter_eig1_5_10,standard_cs_filter_eig2_5_10=epoch_cs_filter_eig1_5_10[num_t*percent:,:], epoch_cs_filter_eig2_5_10[num_t*percent:,:]
    
    # 10-15Hz
    cs_filter_para_eig1_10_15=cs_filter_10_15[ind_eig_cs1_10_15,:]
    cs_filter_para_eig2_10_15=cs_filter_10_15[ind_eig_cs2_10_15,:]
    epoch_cs_filter_eig1_10_15=signal_projection(cs_filter_para_eig1_10_15, epoch_train_10_15[:,300:500,:])
    epoch_cs_filter_eig2_10_15=signal_projection(cs_filter_para_eig2_0_5, epoch_train_10_15[:,300:500,:])
    target_cs_filter_eig1_10_15,target_cs_filter_eig2_10_15=epoch_cs_filter_eig1_10_15[0:num_t*percent,:], epoch_cs_filter_eig2_10_15[0:num_t*percent,:]
    standard_cs_filter_eig1_10_15,standard_cs_filter_eig2_10_15=epoch_cs_filter_eig1_10_15[num_t*percent:,:], epoch_cs_filter_eig2_10_15[num_t*percent:,:]
    

    
    

    # CTP feature
    # 0-5Hz
    epoch_ct_filter_eig1_0_5=signal_projection(ct_filter_0_5[ind_eig_ct1_0_5,:], transpose(epoch_train_0_5[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_0_5=signal_projection(ct_filter_0_5[ind_eig_ct2_0_5,:], transpose(epoch_train_0_5[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_0_5,target_ct_filter_eig2_0_5=epoch_ct_filter_eig1_0_5[0:num_t*percent,:], epoch_ct_filter_eig2_0_5[0:num_t*percent,:]
    standard_ct_filter_eig1_0_5,standard_ct_filter_eig2_0_5=epoch_ct_filter_eig1_0_5[num_t*percent:,:], epoch_ct_filter_eig2_0_5[num_t*percent:,:]
    
    
    # 5-10Hz
    epoch_ct_filter_eig1_5_10=signal_projection(ct_filter_5_10[ind_eig_ct1_5_10,:], transpose(epoch_train_5_10[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_5_10=signal_projection(ct_filter_5_10[ind_eig_ct2_5_10,:], transpose(epoch_train_5_10[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_5_10,target_ct_filter_eig2_5_10=epoch_ct_filter_eig1_5_10[0:num_t*percent,:], epoch_ct_filter_eig2_5_10[0:num_t*percent,:]
    standard_ct_filter_eig1_5_10,standard_ct_filter_eig2_5_10=epoch_ct_filter_eig1_5_10[num_t*percent:,:], epoch_ct_filter_eig2_5_10[num_t*percent:,:]
    
    
    # 10-15Hz
    epoch_ct_filter_eig1_10_15=signal_projection(ct_filter_10_15[ind_eig_ct1_10_15,:], transpose(epoch_train_10_15[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_10_15=signal_projection(ct_filter_10_15[ind_eig_ct2_10_15,:], transpose(epoch_train_10_15[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_10_15,target_ct_filter_eig2_10_15=epoch_ct_filter_eig1_10_15[0:num_t*percent,:], epoch_ct_filter_eig2_10_15[0:num_t*percent,:]
    standard_ct_filter_eig1_10_15,standard_ct_filter_eig2_10_15=epoch_ct_filter_eig1_10_15[num_t*percent:,:], epoch_ct_filter_eig2_10_15[num_t*percent:,:]
    
        


    target_feature_train=c_[log_var(target_cs_filter_eig1_0_5[:,:]),\
                                                log_var(target_cs_filter_eig2_0_5[:,:]),\
                                                log_var(target_ct_filter_eig1_0_5),\
                                                log_var(target_ct_filter_eig2_0_5),\
                                                
                                                log_var(target_cs_filter_eig1_5_10[:,:]),\
                                                log_var(target_cs_filter_eig2_5_10[:,:]),\
                                                log_var(target_ct_filter_eig1_5_10),\
                                                log_var(target_ct_filter_eig2_5_10),\
                                                
                                                log_var(target_cs_filter_eig1_10_15[:,:]),\
                                                log_var(target_cs_filter_eig2_10_15[:,:]),\
                                                log_var(target_ct_filter_eig1_10_15),\
                                                log_var(target_ct_filter_eig2_10_15)]
                                                

    standard_feature_train=c_[log_var(standard_cs_filter_eig1_0_5[:,:]),\
                                                log_var(standard_cs_filter_eig2_0_5[:,:]),\
                                                log_var(standard_ct_filter_eig1_0_5),\
                                                log_var(standard_ct_filter_eig2_0_5),\
                                                
                                                log_var(standard_cs_filter_eig1_5_10[:,:]),\
                                                log_var(standard_cs_filter_eig2_5_10[:,:]),\
                                                log_var(standard_ct_filter_eig1_5_10),\
                                                log_var(standard_ct_filter_eig2_5_10),\
                                                
                                                log_var(standard_cs_filter_eig1_10_15[:,:]),\
                                                log_var(standard_cs_filter_eig2_10_15[:,:]),\
                                                log_var(standard_ct_filter_eig1_10_15),\
                                                log_var(standard_ct_filter_eig2_10_15)]
                                                


    



    standard_feature_train_portion,standard_feature_test_portion=cross_validation.train_test_split(standard_feature_train, train_size=portion,random_state=42)
    
    label_train_eig=r_[ones([shape(target_feature_train)[0],1]),zeros([shape(standard_feature_train_portion[:,:])[0],1])]
    
    X_train_eig=r_[target_feature_train,standard_feature_train_portion[:,:]]
    





    

    #------------------------------------------------classification------------------------------------------
    # trianning LDA
    w_lda,w_lda0=my_new_lda(X_train_eig,label_train_eig)
    lda_project.append(lda_out(w_lda,X_train_eig)[:,0])
    out_lda=lda_predict(X_train_eig,label_train_eig,w_lda,w_lda0)
    TP_lda, FP_lda, TN_lda, FN_lda, P_lda, N_lda=confusion_matrix(out_lda,num_t*percent)
    target_AC_lda_train.append(TP_lda/(TP_lda+FN_lda))
    standard_AC_lda_train.append(TN_lda/(TN_lda+FP_lda))
    BA_lda_train.append((target_AC_lda_train[i]+standard_AC_lda_train[i])/2)

    
    
    # training SVM
    class_SVM=float(num_s)/num_t
    class_SVM=float(num_s*portion)/num_t
    svm_model=svm.LinearSVC(C=.1,class_weight={1:class_SVM})
    svm_model.fit(X_train_eig,label_train_eig)
    out_svm=svm_model.predict(X_train_eig)
    TP_svm, FP_svm, TN_svm, FN_svm, P_svm, N_svm=confusion_matrix(out_svm,num_t*percent)
    target_AC_svm_train.append(TP_svm/(TP_svm+FN_svm))
    standard_AC_svm_train.append(TN_svm/(TN_svm+FP_svm))
    BA_svm_train.append((target_AC_svm_train[i]+standard_AC_svm_train[i])/2)
    
    

    # trainning lda-bayes learning
    pstar_lda=float(num_t)/(float(num_s*portion)+float(num_t))
    w_lda,w_lda0=my_new_lda(X_train_eig,label_train_eig)
    x_lda_out=lda_out(w_lda, X_train_eig)
    out_bayes=bayes_predict(x_lda_out, label_train_eig, x_lda_out,label_train_eig,pstar_lda)
    TP_bayes, FP_bayes, TN_bayes, FN_bayes, P_bayes, N_bayes=confusion_matrix(out_bayes,num_t*percent)
    target_AC_bayes_train.append(TP_bayes/(TP_bayes+FN_bayes))
    standard_AC_bayes_train.append(TN_bayes/(TN_bayes+FP_bayes))
    BA_bayes_train.append((target_AC_bayes_train[i]+standard_AC_bayes_train[i])/2)
    
    
    # -------------------------------------------CSTP testing feature selection-----------------------------------
    # CSP feature
    # 0-5Hz
    cs_filter_para_eig1_0_5_test=cs_filter_0_5[ind_eig_cs1_0_5,:]
    cs_filter_para_eig2_0_5_test=cs_filter_0_5[ind_eig_cs2_0_5,:]
    epoch_cs_filter_eig1_0_5_test=signal_projection(cs_filter_para_eig1_0_5, epoch_test_0_5[:,300:500,:])
    epoch_cs_filter_eig2_0_5_test=signal_projection(cs_filter_para_eig2_0_5, epoch_test_0_5[:,300:500,:])
    target_cs_filter_eig1_0_5_test,target_cs_filter_eig2_0_5_test=epoch_cs_filter_eig1_0_5_test[0:round(num_t*(1-percent)),:], epoch_cs_filter_eig2_0_5_test[0:round(num_t*(1-percent)),:]
    standard_cs_filter_eig1_0_5_test,standard_cs_filter_eig2_0_5_test=epoch_cs_filter_eig1_0_5_test[round(num_t*(1-percent)):,:], epoch_cs_filter_eig2_0_5_test[round(num_t*(1-percent)):,:]
    
    # 5-10Hz
    cs_filter_para_eig1_5_10_test=cs_filter_5_10[ind_eig_cs1_5_10,:]
    cs_filter_para_eig2_5_10_test=cs_filter_5_10[ind_eig_cs2_5_10,:]
    epoch_cs_filter_eig1_5_10_test=signal_projection(cs_filter_para_eig1_5_10_test, epoch_test_5_10[:,300:500,:])
    epoch_cs_filter_eig2_5_10_test=signal_projection(cs_filter_para_eig2_5_10_test, epoch_test_5_10[:,300:500,:])
    target_cs_filter_eig1_5_10_test,target_cs_filter_eig2_5_10_test=epoch_cs_filter_eig1_5_10_test[0:round(num_t*(1-percent)),:], epoch_cs_filter_eig2_5_10_test[0:round(num_t*(1-percent)),:]
    standard_cs_filter_eig1_5_10_test,standard_cs_filter_eig2_5_10_test=epoch_cs_filter_eig1_5_10_test[round(num_t*(1-percent)):,:], epoch_cs_filter_eig2_5_10_test[round(num_t*(1-percent)):,:]
    
    # 10-15Hz
    cs_filter_para_eig1_10_15_test=cs_filter_10_15[ind_eig_cs1_10_15,:]
    cs_filter_para_eig2_10_15_test=cs_filter_10_15[ind_eig_cs2_10_15,:]
    epoch_cs_filter_eig1_10_15_test=signal_projection(cs_filter_para_eig1_10_15_test, epoch_test_10_15[:,300:500,:])
    epoch_cs_filter_eig2_10_15_test=signal_projection(cs_filter_para_eig2_10_15_test, epoch_test_10_15[:,300:500,:])
    target_cs_filter_eig1_10_15_test,target_cs_filter_eig2_10_15_test=epoch_cs_filter_eig1_10_15_test[0:round(num_t*(1-percent)),:], epoch_cs_filter_eig2_10_15_test[0:round(num_t*(1-percent)),:]
    standard_cs_filter_eig1_10_15_test,standard_cs_filter_eig2_10_15_test=epoch_cs_filter_eig1_10_15_test[round(num_t*(1-percent)):,:], epoch_cs_filter_eig2_10_15_test[round(num_t*(1-percent)):,:]
    

    
    

    # CTP feature
    # 0-5Hz
    epoch_ct_filter_eig1_0_5_test=signal_projection(ct_filter_0_5[ind_eig_ct1_0_5,:], transpose(epoch_test_0_5[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_0_5_test=signal_projection(ct_filter_0_5[ind_eig_ct2_0_5,:], transpose(epoch_test_0_5[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_0_5_test,target_ct_filter_eig2_0_5_test=epoch_ct_filter_eig1_0_5_test[0:round(num_t*(1-percent)),:], epoch_ct_filter_eig2_0_5_test[0:round(num_t*(1-percent)),:]
    standard_ct_filter_eig1_0_5_test,standard_ct_filter_eig2_0_5_test=epoch_ct_filter_eig1_0_5_test[round(num_t*(1-percent)):,:], epoch_ct_filter_eig2_0_5_test[round(num_t*(1-percent)):,:]
    
    
    # 5-10Hz
    epoch_ct_filter_eig1_5_10_test=signal_projection(ct_filter_5_10[ind_eig_ct1_5_10,:], transpose(epoch_test_5_10[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_5_10_test=signal_projection(ct_filter_5_10[ind_eig_ct2_5_10,:], transpose(epoch_test_5_10[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_5_10_test,target_ct_filter_eig2_5_10_test=epoch_ct_filter_eig1_5_10_test[0:round(num_t*(1-percent)),:], epoch_ct_filter_eig2_5_10_test[0:round(num_t*(1-percent)),:]
    standard_ct_filter_eig1_5_10_test,standard_ct_filter_eig2_5_10_test=epoch_ct_filter_eig1_5_10_test[round(num_t*(1-percent)):,:], epoch_ct_filter_eig2_5_10_test[round(num_t*(1-percent)):,:]
    
    
    # 10-15Hz
    epoch_ct_filter_eig1_10_15_test=signal_projection(ct_filter_10_15[ind_eig_ct1_10_15,:], transpose(epoch_test_10_15[:,300:500:downsampling,:],(0,2,1)))
    epoch_ct_filter_eig2_10_15_test=signal_projection(ct_filter_10_15[ind_eig_ct2_10_15,:], transpose(epoch_test_10_15[:,300:500:downsampling,:],(0,2,1)))
    target_ct_filter_eig1_10_15_test,target_ct_filter_eig2_10_15_test=epoch_ct_filter_eig1_10_15_test[0:round(num_t*(1-percent)),:], epoch_ct_filter_eig2_10_15_test[0:round(num_t*(1-percent)),:]
    standard_ct_filter_eig1_10_15_test,standard_ct_filter_eig2_10_15_test=epoch_ct_filter_eig1_10_15_test[round(num_t*(1-percent)):,:], epoch_ct_filter_eig2_10_15_test[round(num_t*(1-percent)):,:]
    
    
     
    



    target_feature_test=c_[log_var(target_cs_filter_eig1_0_5_test[:,:]),\
                                                log_var(target_cs_filter_eig2_0_5_test[:,:]),\
                                                log_var(target_ct_filter_eig1_0_5_test),\
                                                log_var(target_ct_filter_eig2_0_5_test),\
                                                
                                                log_var(target_cs_filter_eig1_5_10_test[:,:]),\
                                                log_var(target_cs_filter_eig2_5_10_test[:,:]),\
                                                log_var(target_ct_filter_eig1_5_10_test),\
                                                log_var(target_ct_filter_eig2_5_10_test),\
                                                
                                                log_var(target_cs_filter_eig1_10_15_test[:,:]),\
                                                log_var(target_cs_filter_eig2_10_15_test[:,:]),\
                                                log_var(target_ct_filter_eig1_10_15_test),\
                                                log_var(target_ct_filter_eig2_10_15_test)]
                                                

    standard_feature_test=c_[log_var(standard_cs_filter_eig1_0_5_test[:,:]),\
                                                log_var(standard_cs_filter_eig2_0_5_test[:,:]),\
                                                log_var(standard_ct_filter_eig1_0_5_test),\
                                                log_var(standard_ct_filter_eig2_0_5_test),\
                                                
                                                log_var(standard_cs_filter_eig1_5_10_test[:,:]),\
                                                log_var(standard_cs_filter_eig2_5_10_test[:,:]),\
                                                log_var(standard_ct_filter_eig1_5_10_test),\
                                                log_var(standard_ct_filter_eig2_5_10_test),\
                                                
                                                log_var(standard_cs_filter_eig1_10_15_test[:,:]),\
                                                log_var(standard_cs_filter_eig2_10_15_test[:,:]),\
                                                log_var(standard_ct_filter_eig1_10_15_test),\
                                                log_var(standard_ct_filter_eig2_10_15_test)]
                                                

    
    label_test_eig=r_[ones([shape(target_feature_test)[0],1]),zeros([shape(standard_feature_test)[0],1])]
    X_test_eig=r_[target_feature_test,standard_feature_test]
    
    # --------------------------------------------------classification testing session-------------------------------------
    # lda
    out_test_lda=lda_predict(X_test_eig,label_test_eig,w_lda,w_lda0)
    TP_lda, FP_lda, TN_lda, FN_lda, P_lda, N_lda=confusion_matrix(out_test_lda,round(num_t*(1-percent)))
    target_AC_lda_test.append(TP_lda/(TP_lda+FN_lda))
    standard_AC_lda_test.append(TN_lda/(TN_lda+FP_lda))
    BA_lda_test.append((target_AC_lda_test[i]+standard_AC_lda_test[i])/2)






    # SVM
    out_test_svm=svm_model.predict(X_test_eig)
    TP_svm, FP_svm, TN_svm, FN_svm, P_svm, N_svm=confusion_matrix(out_test_svm,round(num_t*(1-percent)))
    target_AC_svm_test.append(TP_svm/(TP_svm+FN_svm))
    standard_AC_svm_test.append(TN_svm/(TN_svm+FP_svm))
    BA_svm_test.append((target_AC_svm_test[i]+standard_AC_svm_test[i])/2)


    # lda bayes
    x_lda_out_test=lda_out(w_lda, X_test_eig)
    out_test_bayes=bayes_predict(x_lda_out_test, label_test_eig, x_lda_out,label_train_eig,pstar_lda)
    TP_bayes, FP_bayes, TN_bayes, FN_bayes, P_bayes, N_bayes=confusion_matrix(out_test_bayes,round(num_t*(1-percent)))
    target_AC_bayes_test.append(TP_bayes/(TP_bayes+FN_bayes))
    standard_AC_bayes_test.append(TN_bayes/(TN_bayes+FP_bayes))
    BA_bayes_test.append((target_AC_bayes_test[i]+standard_AC_bayes_test[i])/2)
    print 'dataset:', i+1
print mean(BA_lda_test), mean(BA_svm_test), mean(BA_bayes_test)
print 'downsampling',downsampling

C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:113: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:114: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:116: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:117: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:124: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:125: DeprecationWarning: using a

dataset: 1
dataset: 2
dataset: 3
dataset: 4
dataset: 5
dataset: 6
dataset: 7
dataset: 8
dataset: 9
0.684307992203 0.668908382066 0.68693957115
downsampling 5


C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:285: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:292: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:293: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:300: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:301: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:311: DeprecationWarning: using a

In [11]:
BA_svm_test

[0.7280701754385965,
 0.6657894736842105,
 0.8245614035087719,
 0.5228070175438597,
 0.7684210526315789,
 0.6736842105263158,
 0.6684210526315789,
 0.5719298245614035,
 0.5964912280701755]

In [14]:
# plot different undersampling rates on negative class
x=[ .05 ,.1,.2, .3 , .4, .5, .6, .7, .8, .9, 1.]
lda_BA=array([.661,.663,.668,.667,.672,.674,.676,.679,.677,.685, .684])*100 # (m1+m2)/2
svm_BA=array([.643,.652,.656,.666,.653,.661,.661,.660,.659,.664, .668])*100
bayes_BA=array([.656,.657,.675,.675,.669,.672,.678,.684,.680,.680, .687])*100 # bayes threshould
proportion=arange(1,19,19./11)
# .05 ,.1,
# .661,.663,
# .643,.652,
# .656,.657,
# x=[.05 ,.1, .2, .3 , .4, .5, .6, .7, .8, .9, 1.]
# lda=array([.679,.683,.694,.697,.698,.691,.688,.700,.700,.696, .696])*100 # (m1+m2)/2
# svm=array([.664,.683,.694,.687,.684,.671,.676,.686,.686,.681, .679])*100
# bayes=array([.682,.684,.690,.692,.691,.694,.692,.692,.693,.694, .702])*100 # bayes threshould
# proportion=arange(1,19,19./11)


figure()
# plot(x,linear)

plot(x,lda_BA)
plot(x,svm_BA)
plot(x,bayes_BA)
legend(['LDA','WSVM','LDA-Bayes'], loc=4, borderaxespad=0.)
xticks(arange(min(x), max(x)+1, .1))
xlim([.05,1])
xlabel('Downsampling rate for negative class in training session',fontsize=14,weight='bold')
ylabel('Balanced accuracy (%)',fontsize=14,weight='bold')
title('Balanced accuracy with varying base rate',fontsize=15,weight='bold')
print std(lda_BA), std(svm_BA), std(bayes_BA)
print mean(lda_BA), mean(svm_BA), mean(bayes_BA)





0.758096790523 0.681393807325 0.953895882878
67.3272727273 65.8454545455 67.3909090909


C:\Users\villa_000\Anaconda\lib\site-packages\matplotlib\font_manager.py:1282: UserWarning: findfont: Font family [u'normal'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [12]:
print mean(BA_lda_test[0:5]),mean(BA_svm_test[0:5]),mean(BA_bayes_test[0:5])
print mean(BA_lda_test[5:]),mean(BA_svm_test[5:]),mean(BA_bayes_test[5:])
print mean(BA_lda_test),mean(BA_svm_test), mean(BA_bayes_test)

0.721929824561 0.701929824561 0.722105263158
0.637280701754 0.627631578947 0.64298245614
0.684307992203 0.668908382066 0.68693957115


# LDA threshold selection

In [15]:
# observe mean and std in LDA projected space in training dataset
lda_project_t_mu=[]
lda_project_t_std=[]
lda_project_s_mu=[]
lda_project_s_std=[]
for i in range(0,len(data_sub)):
    variable,chan=shape(load_epoch(data_path[0],'epoch_target'))[1], 32
    dataset=data_sub[i]
    percent=.9  # training rate
    test_percent=1-percent
    portion=.99
    
    epoch_target_dataset=[]
    epoch_standard_dataset=[]
    epoch_target=zeros([1,variable,chan])
    epoch_standard=zeros([1,variable,chan])
    for j in range(0,len(dataset)):
        epoch_target_dataset.append(load_epoch(dataset[j],'epoch_target'))
        epoch_standard_dataset.append(load_epoch(dataset[j],'epoch_standard')) # chan variable obs
        epoch_target_32,epoch_standard_32=epoch_target_dataset[j], epoch_standard_dataset[j]
        epoch_target_6,epoch_standard_6=epoch_target_32[:,:,:],\
                                        epoch_standard_32[:,:,:]
#         epoch_target_6,epoch_standard_6=epoch_target_32[array([2,24,13,14,19,17,16,18])-1,:,:],\
#                                         epoch_standard_32[array([2,24,13,14,19,17,16,18])-1,:,:]
        epoch_target_6,epoch_standard_6=epoch_target_6.T, epoch_standard_6.T    # obs variable chan
        epoch_target=r_[epoch_target, epoch_target_6]
        epoch_standard=r_[epoch_standard, epoch_standard_6]
        epoch_target=epoch_target[1:,:,:]    
        epoch_standard=epoch_standard[1:,:,:]
    flag=shape(epoch_target)[0]+shape(epoch_standard)[0] # total number of epochs
    num_t=shape(epoch_target)[0] # number of target
    num_s=shape(epoch_standard)[0] # number of standard
    epoch_all= r_[epoch_target,epoch_standard]
    
    
    
    lda_project_array=lda_project[i]
    lda_project_t=lda_project_array[0:num_t*percent]
    lda_project_s=lda_project_array[num_t*percent:]
    lda_project_t_mu.append(mean(lda_project_t))
    lda_project_t_std.append(std(lda_project_t))
    lda_project_s_mu.append(mean(lda_project_s))
    lda_project_s_std.append(std(lda_project_s))
    print 'excuting dataset',i+1,'.............'
    
print array(lda_project_t_mu)
print array(lda_project_s_mu)
print array(lda_project_t_std)
print array(lda_project_s_std)

excuting dataset 1 .............
excuting dataset 2 .............
excuting dataset 3 .............
excuting dataset 4 .............
excuting dataset 5 .............
excuting dataset 6 .............
excuting dataset 7 .............
excuting dataset 8 .............
excuting dataset 9 .............
[  0.81268326   5.31306382  15.39678131   3.68351433  11.11165384
   9.25112052  10.92627889  -1.40383539   2.26680628]
[  0.01546827   3.62598777  12.4360933    2.69020231   9.03835874
   7.97439777   9.57508327  -1.83628976   1.58996405]
[ 0.71526946  1.1876712   1.56484997  1.08973347  1.4296488   1.14550783
  1.44559532  0.37104813  0.59979033]
[ 0.80031345  1.22112292  1.69164961  0.93141824  1.33326969  1.07501279
  1.05664964  0.43252228  0.53099867]


C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:37: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
C:\Users\villa_000\Anaconda\lib\site-packages\ipykernel\__main__.py:38: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [16]:
print mean(array(lda_project_t_mu)-
           array(lda_project_s_mu))
print mean(abs(array(lda_project_t_std)-array(lda_project_s_std)))

1.36097790157
0.121077358315


In [19]:
hist((array(lda_project_t_std)-array(lda_project_s_std))/(array(lda_project_t_mu)-array(lda_project_s_mu)),bins=3)
ylabel('Number',fontweight='bold')
xlabel('Difference of standard deviation',fontweight='bold')
title('Normalized difference of standard deviation in two conditions',fontweight='bold',fontsize=13)

# plot CSTP features

In [ ]:
import pandas as pd
from mne.io import RawArray
from mne.channels import read_montage
from mne.epochs import concatenate_epochs
from mne import create_info, find_events, Epochs
from mne.viz.topomap import _prepare_topo_plot, plot_topomap
from mne.decoding import CSP

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import cross_val_score, LeaveOneLabelOut
from glob import glob
import matplotlib.pyplot as plt
from scipy.signal import welch
from mne import pick_types
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
y = []
epochs_tot = []
# os.chdir('/Users/Villa/Desktop/1k@5Hz_Expert_user5/EEG_recordings')
os.chdir('D:\\data\\eeg_data\\eva_EEG\\1k@5Hz_Expert_user5\\EEG_recordings')
montage=read_montage('standard_1020')
data=mne.io.read_raw_brainvision('1_figure.vhdr', preload=True,montage=montage)
picks = pick_types(data.info,eeg=True)
events=find_events(data)
data.filter(1, 20, picks=picks, filter_length='10s', l_trans_bandwidth=0.5, h_trans_bandwidth=0.5, n_jobs=1, method='fft', iir_params=None, verbose=None)
epoch_target=Epochs(data, events, [2],-1, 2, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose =False)
epoch_target.resample(250)


epoch_standard=Epochs(data, events, [1],-1, 2, proj=False,
                    picks=picks, baseline=None, preload=True,
                    add_eeg_ref=False, verbose =False)
epoch_standard.resample(250)



epochs_tot.append(epoch_target)
epochs_tot.append(epoch_standard)
epochs = concatenate_epochs(epochs_tot)
_,epos,_,_,_  = _prepare_topo_plot(epochs,'eeg',None)
y.extend([1]*len(epoch_target))
y.extend([0]*len(epoch_standard))
X = epochs.get_data()
y = np.array(y)


variable,chan=shape(load_epoch(data_path[0],'epoch_target'))[1], 32
dataset=data_sub[0]
percent=.9  # training rate
test_percent=1-percent
portion=.99

epoch_target_dataset=[]
epoch_standard_dataset=[]
epoch_target=zeros([1,variable,chan])
epoch_standard=zeros([1,variable,chan])
for j in range(0,len(dataset)):
    epoch_target_dataset.append(load_epoch(dataset[j],'epoch_target'))
    epoch_standard_dataset.append(load_epoch(dataset[j],'epoch_standard')) # chan variable obs
    epoch_target_32,epoch_standard_32=epoch_target_dataset[j], epoch_standard_dataset[j]
    epoch_target_6,epoch_standard_6=epoch_target_32[:,:,:],\
                                    epoch_standard_32[:,:,:]
#         epoch_target_6,epoch_standard_6=epoch_target_32[array([2,24,13,14,19,17,16,18])-1,:,:],\
#                                         epoch_standard_32[array([2,24,13,14,19,17,16,18])-1,:,:]
    epoch_target_6,epoch_standard_6=epoch_target_6.T, epoch_standard_6.T    # obs variable chan
    epoch_target=r_[epoch_target, epoch_target_6]
    epoch_standard=r_[epoch_standard, epoch_standard_6]
    epoch_target=epoch_target[1:,:,:]    
    epoch_standard=epoch_standard[1:,:,:]

flag=shape(epoch_target)[0]+shape(epoch_standard)[0] # total number of epochs
num_t=shape(epoch_target)[0] # number of target
num_s=shape(epoch_standard)[0] # number of standard
epoch_all= r_[epoch_target,epoch_standard]
#     epoch_all   # r_[target, standard]

#   ----------------------------------------------filter-------------------------------------
#   0-5Hz  
epoch_all_0_5=zeros_like(epoch_all)
for j in range(0,chan):
    epoch_all_0_5[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], .1, 5, 250, order=3)


#   5-10Hz  
epoch_all_5_10=zeros_like(epoch_all)
for j in range(0,chan):
    epoch_all_5_10[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], 5, 10, 250, order=3)   

#   10-15Hz  
epoch_all_10_15=zeros_like(epoch_all)
for j in range(0,chan):
    epoch_all_10_15[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], 10, 15, 250, order=3)    

# #   15-20Hz  
#     epoch_all_15_20=zeros_like(epoch_all)
#     for j in range(0,chan):
#         epoch_all_15_20[:,:,j]=butter_bandpass_filter(epoch_all[:,:,j], 15, 20, 250, order=3)

#   --------------------------------------------construct sub band training and testing epochs for CSTP--------------------------------
# 0-5Hz
epoch_target_train_0_5, epoch_target_test_0_5=cross_validation.train_test_split(epoch_all_0_5[0:num_t,:,:], train_size=percent,random_state=42)
epoch_standard_train_0_5, epoch_standard_test_0_5=cross_validation.train_test_split(epoch_all_0_5[num_t:,:,:], train_size=percent,random_state=42)
epoch_train_0_5=r_[epoch_target_train_0_5,epoch_standard_train_0_5]
label_train=r_[ones(shape(epoch_target_train_0_5)[0]),zeros(shape(epoch_standard_train_0_5)[0])]
epoch_test_0_5=r_[epoch_target_test_0_5,epoch_standard_test_0_5]
label_test=r_[ones(shape(epoch_target_test_0_5)[0]),zeros(shape(epoch_standard_test_0_5)[0])]

# 5-10Hz
epoch_target_train_5_10, epoch_target_test_5_10=cross_validation.train_test_split(epoch_all_5_10[0:num_t,:,:], train_size=percent,random_state=42)
epoch_standard_train_5_10, epoch_standard_test_5_10=cross_validation.train_test_split(epoch_all_5_10[num_t:,:,:], train_size=percent,random_state=42)
epoch_train_5_10=r_[epoch_target_train_5_10,epoch_standard_train_5_10]
epoch_test_5_10=r_[epoch_target_test_5_10,epoch_standard_test_5_10]

# 10-15Hz
epoch_target_train_10_15, epoch_target_test_10_15=cross_validation.train_test_split(epoch_all_10_15[0:num_t,:,:], train_size=percent,random_state=42)
epoch_standard_train_10_15, epoch_standard_test_10_15=cross_validation.train_test_split(epoch_all_10_15[num_t:,:,:], train_size=percent,random_state=42)
epoch_train_10_15=r_[epoch_target_train_10_15,epoch_standard_train_10_15]
epoch_test_10_15=r_[epoch_target_test_10_15,epoch_standard_test_10_15]

#     # 15-20Hz
#     epoch_target_train_15_20, epoch_target_test_15_20=cross_validation.train_test_split(epoch_all_15_20[0:num_t,:,:], train_size=percent,random_state=42)
#     epoch_standard_train_15_20, epoch_standard_test_15_20=cross_validation.train_test_split(epoch_all_15_20[num_t:,:,:], train_size=percent,random_state=42)
#     epoch_train_15_20=r_[epoch_target_train_15_20,epoch_standard_train_15_20]
#     epoch_test_15_20=r_[epoch_target_test_15_20,epoch_standard_test_15_20]


# ---------------------------------------------CSTP calculation----------------------------------------
# train 300-500
downsampling=5
# 0-5Hz
cs_pattern_0_5,cs_filter_0_5=myCSP(epoch_train_0_5[:,300:500,:],label_train)
ct_pattern_0_5, ct_filter_0_5=myCTP(epoch_train_0_5[:,300:500,:],label_train,downsampling)

ind_eig_cs1_0_5=max_target_lamba_ind(epoch_train_0_5[0:num_t*percent,300:500,:],epoch_train_0_5[num_t*percent:,300:500,:],cs_pattern_0_5,cs_filter_0_5)
ind_eig_ct1_0_5=max_target_lamba_ind(transpose(epoch_train_0_5[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_0_5[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_0_5,ct_filter_0_5)

ind_eig_cs2_0_5=max_target_lamba_ind(epoch_train_0_5[num_t*percent:,300:500,:],epoch_train_0_5[0:num_t*percent,300:500,:],cs_pattern_0_5,cs_filter_0_5)
ind_eig_ct2_0_5=max_target_lamba_ind(transpose(epoch_train_0_5[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_0_5[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_0_5,ct_filter_0_5)


# 5-10Hz
cs_pattern_5_10,cs_filter_5_10=myCSP(epoch_train_5_10[:,300:500,:],label_train)
ct_pattern_5_10, ct_filter_5_10=myCTP(epoch_train_5_10[:,300:500,:],label_train,downsampling)

ind_eig_cs1_5_10=max_target_lamba_ind\
                (epoch_train_5_10[0:num_t*percent,300:500,:],epoch_train_5_10[num_t*percent:,300:500,:],cs_pattern_5_10,cs_filter_5_10)
ind_eig_ct1_5_10=max_target_lamba_ind(transpose(epoch_train_5_10[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_5_10[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_5_10,ct_filter_5_10)

ind_eig_cs2_5_10=max_target_lamba_ind\
                (epoch_train_5_10[num_t*percent:,300:500,:],epoch_train_5_10[0:num_t*percent,300:500,:],cs_pattern_5_10,cs_filter_5_10)
ind_eig_ct2_5_10=max_target_lamba_ind(transpose(epoch_train_5_10[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_5_10[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_5_10,ct_filter_5_10)


# 10-15Hz
cs_pattern_10_15,cs_filter_10_15=myCSP(epoch_train_10_15[:,300:500,:],label_train)
ct_pattern_10_15, ct_filter_10_15=myCTP(epoch_train_10_15[:,300:500,:],label_train,downsampling)

ind_eig_cs1_10_15=max_target_lamba_ind\
                  (epoch_train_10_15[0:num_t*percent,300:500,:],epoch_train_10_15[num_t*percent:,300:500,:],cs_pattern_10_15,cs_filter_10_15)
ind_eig_ct1_10_15=max_target_lamba_ind(transpose(epoch_train_10_15[0:num_t*percent,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_10_15[num_t*percent:,300:500:downsampling,:],(0,2,1)),ct_pattern_10_15,ct_filter_10_15)

ind_eig_cs2_10_15=max_target_lamba_ind\
                 (epoch_train_10_15[num_t*percent:,300:500,:],epoch_train_10_15[0:num_t*percent,300:500,:],cs_pattern_10_15,cs_filter_10_15)
ind_eig_ct2_10_15=max_target_lamba_ind(transpose(epoch_train_10_15[num_t*percent:,300:500:downsampling,:],(0,2,1)),\
                              transpose(epoch_train_10_15[0:num_t*percent,300:500:downsampling,:],(0,2,1)),ct_pattern_10_15,ct_filter_10_15)


# -------------------------------------------CSTP training feature selection-----------------------------------
# CSP feature
# 0-5Hz
cs_filter_para_eig1_0_5=cs_filter_0_5[ind_eig_cs1_0_5,:]
cs_filter_para_eig2_0_5=cs_filter_0_5[ind_eig_cs2_0_5,:]
epoch_cs_filter_eig1_0_5=signal_projection(cs_filter_para_eig1_0_5, epoch_train_0_5[:,300:500,:])
epoch_cs_filter_eig2_0_5=signal_projection(cs_filter_para_eig2_0_5, epoch_train_0_5[:,300:500,:])
target_cs_filter_eig1_0_5,target_cs_filter_eig2_0_5=epoch_cs_filter_eig1_0_5[0:num_t*percent,:], epoch_cs_filter_eig2_0_5[0:num_t*percent,:]
standard_cs_filter_eig1_0_5,standard_cs_filter_eig2_0_5=epoch_cs_filter_eig1_0_5[num_t*percent:,:], epoch_cs_filter_eig2_0_5[num_t*percent:,:]

# 5-10Hz
cs_filter_para_eig1_5_10=cs_filter_5_10[ind_eig_cs1_5_10,:]
cs_filter_para_eig2_5_10=cs_filter_5_10[ind_eig_cs2_5_10,:]
epoch_cs_filter_eig1_5_10=signal_projection(cs_filter_para_eig1_5_10, epoch_train_5_10[:,300:500,:])
epoch_cs_filter_eig2_5_10=signal_projection(cs_filter_para_eig2_5_10, epoch_train_5_10[:,300:500,:])
target_cs_filter_eig1_5_10,target_cs_filter_eig2_5_10=epoch_cs_filter_eig1_5_10[0:num_t*percent,:], epoch_cs_filter_eig2_5_10[0:num_t*percent,:]
standard_cs_filter_eig1_5_10,standard_cs_filter_eig2_5_10=epoch_cs_filter_eig1_5_10[num_t*percent:,:], epoch_cs_filter_eig2_5_10[num_t*percent:,:]

# 10-15Hz
cs_filter_para_eig1_10_15=cs_filter_10_15[ind_eig_cs1_10_15,:]
cs_filter_para_eig2_10_15=cs_filter_10_15[ind_eig_cs2_10_15,:]
epoch_cs_filter_eig1_10_15=signal_projection(cs_filter_para_eig1_10_15, epoch_train_10_15[:,300:500,:])
epoch_cs_filter_eig2_10_15=signal_projection(cs_filter_para_eig2_0_5, epoch_train_10_15[:,300:500,:])
target_cs_filter_eig1_10_15,target_cs_filter_eig2_10_15=epoch_cs_filter_eig1_10_15[0:num_t*percent,:], epoch_cs_filter_eig2_10_15[0:num_t*percent,:]
standard_cs_filter_eig1_10_15,standard_cs_filter_eig2_10_15=epoch_cs_filter_eig1_10_15[num_t*percent:,:], epoch_cs_filter_eig2_10_15[num_t*percent:,:]



# CTP feature
# 0-5Hz
epoch_ct_filter_eig1_0_5=signal_projection(ct_filter_0_5[ind_eig_ct1_0_5,:], transpose(epoch_train_0_5[:,300:500:downsampling,:],(0,2,1)))
epoch_ct_filter_eig2_0_5=signal_projection(ct_filter_0_5[ind_eig_ct2_0_5,:], transpose(epoch_train_0_5[:,300:500:downsampling,:],(0,2,1)))
target_ct_filter_eig1_0_5,target_ct_filter_eig2_0_5=epoch_ct_filter_eig1_0_5[0:num_t*percent,:], epoch_ct_filter_eig2_0_5[0:num_t*percent,:]
standard_ct_filter_eig1_0_5,standard_ct_filter_eig2_0_5=epoch_ct_filter_eig1_0_5[num_t*percent:,:], epoch_ct_filter_eig2_0_5[num_t*percent:,:]


# 5-10Hz
epoch_ct_filter_eig1_5_10=signal_projection(ct_filter_5_10[ind_eig_ct1_5_10,:], transpose(epoch_train_5_10[:,300:500:downsampling,:],(0,2,1)))
epoch_ct_filter_eig2_5_10=signal_projection(ct_filter_5_10[ind_eig_ct2_5_10,:], transpose(epoch_train_5_10[:,300:500:downsampling,:],(0,2,1)))
target_ct_filter_eig1_5_10,target_ct_filter_eig2_5_10=epoch_ct_filter_eig1_5_10[0:num_t*percent,:], epoch_ct_filter_eig2_5_10[0:num_t*percent,:]
standard_ct_filter_eig1_5_10,standard_ct_filter_eig2_5_10=epoch_ct_filter_eig1_5_10[num_t*percent:,:], epoch_ct_filter_eig2_5_10[num_t*percent:,:]


# 10-15Hz
epoch_ct_filter_eig1_10_15=signal_projection(ct_filter_10_15[ind_eig_ct1_10_15,:], transpose(epoch_train_10_15[:,300:500:downsampling,:],(0,2,1)))
epoch_ct_filter_eig2_10_15=signal_projection(ct_filter_10_15[ind_eig_ct2_10_15,:], transpose(epoch_train_10_15[:,300:500:downsampling,:],(0,2,1)))
target_ct_filter_eig1_10_15,target_ct_filter_eig2_10_15=epoch_ct_filter_eig1_10_15[0:num_t*percent,:], epoch_ct_filter_eig2_10_15[0:num_t*percent,:]
standard_ct_filter_eig1_10_15,standard_ct_filter_eig2_10_15=epoch_ct_filter_eig1_10_15[num_t*percent:,:], epoch_ct_filter_eig2_10_15[num_t*percent:,:]





In [ ]:
# plot spatial filtered signal and temporal pattern
figure()
subplot(2,2,1)
ts=arange(200,1000,4)
plot(ts,mean(target_cs_filter_eig1_0_5,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Target features')
title('Filtered target temporal features',fontweight='bold',fontsize=25)

subplot(2,2,3)
plot(ts,mean(target_cs_filter_eig2_0_5,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered target temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)

subplot(2,2,2)
plot(ts,mean(standard_cs_filter_eig1_0_5,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Standard features')
title('Filtered standard temporal features',fontweight='bold',fontsize=25)

subplot(2,2,4)
plot(ts,mean(standard_cs_filter_eig2_0_5,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered standard temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)



fig,ax_topo = subplots(1,1)
img, _ = plot_topomap(mean(target_ct_filter_eig1_0_5,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(-mean(target_ct_filter_eig2_0_5,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 2',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(mean(standard_ct_filter_eig1_0_5,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(-mean(standard_ct_filter_eig2_0_5,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 2',fontweight='bold',fontsize=21)


# figure()
# subplot(2,1,1)
# ts=arange(200,1000,4)
# plot(ts,-mean(target_cs_filter_eig1,axis=0),'r',linewidth=2)
# plot(ts,mean(standard_cs_filter_eig1,axis=0),'b',linewidth=2)
# legend(['Target', 'Standard'])
# xlabel('Time (ms)')
# title('Spatial filtered signals')

# # subplot(2,1,2)
# # tt=arange(200,1000,20)
# # plot(tt,ct_pattern[:,ind_eig_ct1],'r',linewidth=2)
# # plot(tt,ct_pattern[:,ind_eig_ct2],'b',linewidth=2)
# # legend(['Target', 'Standard'])
# # xlabel('Time (ms)')
# # title('Common temporal patterns')


# figure()
# ts=arange(200,1000,4)
# plot(ts,-mean(target_cs_filter_eig1,axis=0),'r',linewidth=2)
# plot(ts,mean(epoch_filter[0:num_t,300:500,13],axis=0),'b',linewidth=2)
# legend(['Spatial filtered target condition', 'Non-spatial filtered target condition'])
# xlabel('Time (ms)')

In [ ]:
# plot spatial filtered signal and temporal pattern
figure()

subplot(2,2,1)
ts=arange(200,1000,4)
plot(ts,mean(target_cs_filter_eig1_5_10,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Target features')
title('Filtered target temporal features',fontweight='bold',fontsize=25)

subplot(2,2,3)
plot(ts,mean(target_cs_filter_eig2_5_10,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered target temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)

subplot(2,2,2)
plot(ts,mean(standard_cs_filter_eig1_5_10,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Standard features')
title('Filtered standard temporal features',fontweight='bold',fontsize=25)

subplot(2,2,4)
plot(ts,mean(standard_cs_filter_eig2_5_10,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered standard temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)



fig,ax_topo = subplots(1,1)
img, _ = plot_topomap(-mean(target_ct_filter_eig1_5_10,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(-mean(target_ct_filter_eig2_5_10,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 2',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(mean(standard_ct_filter_eig1_5_10,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(mean(-standard_ct_filter_eig2_5_10,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 2',fontweight='bold',fontsize=21)

In [ ]:
# plot spatial filtered signal and temporal pattern
figure()
subplot(2,2,1)
ts=arange(200,1000,4)
plot(ts,mean(target_cs_filter_eig1_10_15,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Target features')
title('Filtered target temporal features',fontweight='bold',fontsize=25)

subplot(2,2,3)
plot(ts,mean(target_cs_filter_eig2_10_15,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered target temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)

subplot(2,2,2)
plot(ts,mean(standard_cs_filter_eig1_10_15,axis=0))
xlabel('Time (ms)',fontweight='bold')
title('Standard features')
title('Filtered standard temporal features',fontweight='bold',fontsize=25)

subplot(2,2,4)
plot(ts,mean(standard_cs_filter_eig2_10_15,axis=0))
xlabel('Time (ms)',fontweight='bold')
# title('Filtered standard temporal features by standard condition spatial filter',fontweight='bold',fontsize=21)



fig,ax_topo = subplots(1,1)
img, _ = plot_topomap(-mean(target_ct_filter_eig1_10_15,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(-mean(target_ct_filter_eig2_10_15,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered target spatial features by temporal filter 2',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(mean(standard_ct_filter_eig1_10_15,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 1',fontweight='bold',fontsize=21)


fig,ax_topo = subplots(1, 1)
img, _ = plot_topomap(-mean(standard_ct_filter_eig2_10_15,axis=0),epos)
divider = make_axes_locatable(ax_topo)
ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
colorbar(img, cax=ax_colorbar)
# suptitle('Filtered standard spatial features by temporal filter 2',fontweight='bold',fontsize=21)